In [1]:
import sys
del sys.path[5]
del sys.path[5]
sys.path.insert(0,'/home/pgaemers/software/epix')
sys.path.insert(0,'/home/pgaemers/software/wfsim')

In [2]:
import numpy as np
import straxen
import epix
from epix.simulator.fast_simulator import StraxSimulator

print(f'epix version : {epix.__version__}')

Using nestpy version 1.3.0
epix version : 0.1.1


In [3]:
# MC output file as an input for epix/fast_simulator
raw_data_dir='/dali/lgrandi/xenonnt/simulations/testing_epix_wfsim/'
raw_data_filename='tpc_and_nveto_cryoneutrons_200.root'
st = straxen.contexts.xenonnt_simulation(output_folder='/dali/lgrandi/pgaemers/strax_data_fast_sim')
run_id = '3'
st.register(StraxSimulator)

epix.simulator.fast_simulator.StraxSimulator

In [4]:
# %rm -r './strax_data'
fax_config='fax_config_nt_design.json'

# epix config
epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':100000,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.005,
           'tag_cluster_by':'time',
           'max_delay':1e-7,
           'source_rate':-1}

# configuration files; to be moved into the plugin
configuration_files={'nv_pmt_qe':'nveto_pmt_qe.json',
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),}

st.config.update(dict(fax_config=fax_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      configuration_files=configuration_files,
                      fax_config_overrides= {'detector':'XENONnT',
                                             "enable_field_dependencies": {
                                            "survival_probability_map": False,
                                            "drift_speed_map": False,
                                            "diffusion_longitudinal_map": False,
                                            "diffusion_radial_map": False,
                                            "diffusion_azimuthal_map": False}}))

In [5]:
st.register(StraxSimulator)

epix.simulator.fast_simulator.StraxSimulator

In [6]:
st.make(run_id,'events_tpc')

epix configuration:  {'path': '/dali/lgrandi/xenonnt/simulations/testing_epix_wfsim/', 'file_name': 'tpc_and_nveto_cryoneutrons_200.root', 'debug': True, 'entry_start': 0, 'entry_stop': 100000, 'cut_by_eventid': False, 'micro_separation_time': 10.0, 'micro_separation': 0.005, 'tag_cluster_by': 'time', 'max_delay': 1e-07, 'source_rate': -1, 'detector_config': [<epix.detector_volumes.SensitiveVolume object at 0x7f073c8b5a30>, <epix.detector_volumes.SensitiveVolume object at 0x7f073c8b54f0>], 'outer_cylinder': {'max_z': 7.3936, 'min_z': -154.6555, 'max_r': 66.4}}
Total entries in input file = 198
Starting to read from output file entry 0
Ending read in at output file entry 100000
It took 1.6784 sec to load data.
Finding clusters of interactions with a dr = 0.005 mm and dt = 10.0 ns
It took 3.5242 sec to find clusters.
It took 3.7205 sec to merge clusters.
Removing clusters not in volumes: TPC BelowCathode
Number of clusters before: 6120
Number of clusters after: 6047
Assigning electric fi

/home/pgaemers/software/epix/epix/quanta_generation.py:51: UserWarning: Energy deposition of 206.56977462768555 keV beyond NEST validity for NR model of 200 keV - Remove Interaction
  warnings.warn(f"Energy deposition of {en} keV beyond NEST validity for NR model of 200 keV - Remove Interaction")
/home/pgaemers/software/epix/epix/quanta_generation.py:51: UserWarning: Energy deposition of 279.19512462615967 keV beyond NEST validity for NR model of 200 keV - Remove Interaction
  warnings.warn(f"Energy deposition of {en} keV beyond NEST validity for NR model of 200 keV - Remove Interaction")
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2192: RuntimeWarning: overflow encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.8/site-packages/numpy/lib/function_base.py:2192: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


Source finished!


In [7]:
# @numba.njit
# def _merge_these_clusters(s2_area1, z1, s2_area2, z2):
#     sensitive_volume_ztop=0 #it's the ground mesh, the top liquid level is at 2.7; // mm
#     max_s2_area = max(s2_area1,s2_area2)
#     if max_s2_area>5000:
#         SeparationDistanceIntercept = 0.00024787 * 5000. + 3.4056346550312973
#         SeparationDistanceSlope = 5.5869678412887262e-07 * 5000. + 0.0044792968
#     else:
#         SeparationDistanceIntercept = \
#           0.00024787 * max_s2_area + 3.4056346550312973
#         SeparationDistanceSlope = \
#           5.5869678412887262e-07 * max_s2_area + 0.0044792968
#     SeparationDistance = \
#         SeparationDistanceIntercept - \
#         SeparationDistanceSlope * (-sensitive_volume_ztop +\
#             (z1 + z2) * 0.5)
#     return z1-z2<SeparationDistance

In [8]:
# @numba.njit
# def _macro_cluster_events(instructions):
#     for ix1 in range(len(instructions)):
# #         print(f"ix1)
#         if instructions[ix1]['type']!=2:
#             continue
#         for ix2 in range(1,len(instructions[ix1:])+1):
#             print(ix1+ix2)
#             if instructions[ix1+ix2]['type']!=2:
#                 continue
#             if instructions[ix1]['event_number']!=instructions[ix1+ix2]['event_number']:
#                 break
#             if _merge_these_clusters(instructions[ix1]['amp'],instructions[ix1]['z'],
#                                      instructions[ix1+ix2]['amp'],instructions[ix1+ix2]['z']):
#                 print('Merging!')
#                 instructions[ix1+ix2]['x'] = (instructions[ix1]['x']+instructions[ix1+ix2]['x'])*0.5
#                 instructions[ix1+ix2]['y'] = (instructions[ix1]['y']+instructions[ix1+ix2]['y'])*0.5
#                 instructions[ix1+ix2]['z'] = (instructions[ix1]['z']+instructions[ix1+ix2]['z'])*0.5
#                 instructions[ix1+ix2]['amp'] = int((instructions[ix1]['amp']+instructions[ix1+ix2]['amp'])*0.5)
#                 instructions[ix1]['amp']=-1 #flag to throw this instruction away later